## AGIにおける計算の各部分のベンチマークを計測する。

|計測部分|一回あたりの実行時間|コード内の計算回数|
|:-|:-|:-|
|旧lambdify|70.83559119701385[sec]|ドラッグの度|
|新lambdify|3.4351510787010193[sec]|1回のみ|
|opt.minimize|0.04132338047027588[sec]|ドラッグの度|
|基底更新|1.6651153564453126e-05[sec]|ドラッグの度|
|座標再計算|3.598928451538086e-05[sec]|ドラッグの度|
|図のフィールド更新|0.00024055957794189454[sec]|ドラッグの度|

In [4]:
import numpy as np
import sympy as sp
from scipy import optimize as opt
from matplotlib import pyplot as plt
from sympy.utilities.lambdify import lambdify
from sympy import Matrix

#  initialize
edge = np.genfromtxt('csv/adjacency.csv', delimiter=",").astype(np.int64)
P = np.genfromtxt('csv/mdSpace.csv', delimiter=",")
n = len(P)
L = np.genfromtxt('csv/eigVals.csv', delimiter=",")
L_pos = np.array([L[i] if L[i] > 0 else 0 for i in range(n)])
d = np.count_nonzero(L_pos)
Ln = np.sqrt(L_pos)
f2 = np.array(Ln[0:d])
f2[::2] = 0
f1 = Ln[0:d] - f2
e1 = (f1 / np.linalg.norm(f1)).reshape(d,1)
e2 = (f2 / np.linalg.norm(f2)).reshape(d,1)
temp1 = e1
temp2 = e2

print(P[0].shape)
print(type(P[0]))

Xs = np.zeros(n)
Ys = np.zeros(n)

def update_points():
	for i in np.arange(n):
		global Xs, Ys
		p0 = P[i, 0:d]
		Xs[i] = np.dot(p0, e1)
		Ys[i] = np.dot(p0, e2)

update_points()

print("init: ready")

# sympy
a1,b1,c1,a2,b2,c2,t,s = sp.symbols('a1 b1 c1 a2 b2 c2 t s')   # variables
x2_s,y2_s = sp.symbols('x2_s y2_s')  # values
P_i = sp.MatrixSymbol('P_i', d, 1)
E1 = sp.MatrixSymbol('E1', d, 1)
E2 = sp.MatrixSymbol('E2', d, 1)
var = (x2_s,y2_s,P_i,E1,E2,a1,b1,c1,a2,b2,c2,t,s)

_E1 = a1*sp.Matrix(E1) + b1*sp.Matrix(E2) + c1*sp.Matrix(P_i)
_E2 = a2*sp.Matrix(E1) + b2*sp.Matrix(E2) + c2*sp.Matrix(P_i)
R = s*sp.Matrix(E1) + t*sp.Matrix(E2)

f = Matrix([
		_E1.dot(_E1) - 1,
		_E2.dot(_E2) - 1,
		_E1.dot(_E2),
		R.dot(R) - 1,
		_E1.dot(R) - sp.Matrix(E1).dot(R),
		_E2.dot(R) - sp.Matrix(E2).dot(R),
		sp.Matrix(P_i).dot(_E1) - x2_s,
		sp.Matrix(P_i).dot(_E2) - y2_s
		])

func = sp.Matrix.norm(f)

lam_f = lambdify(var, func, 'numpy')

def lam(x2, y2, p, e_1, e_2):
    return lambda a1,b1,c1,a2,b2,c2,t,s: \
        lam_f(x2, y2, p, e_1, e_2, a1, b1, c1, a2, b2, c2, t, s)

print("lambda: ready")

######## Graph Drawing ########
identifier = ""
arr = np.array([1, 1, 1, 1, 1, 1, 1, 1])

gca = plt.gca()
nodes = np.array([])
edges = np.array([])

for i in np.arange(n):
	circle = plt.Circle((Xs[i], Ys[i]), radius=0.2, fc='y', label=str(i))
	gca.add_patch(circle)
	nodes = np.append(nodes, circle)

for e in edge:
	line = plt.Line2D((Xs[e[0] - 1], Xs[e[1] - 1]), (Ys[e[0] - 1], Ys[e[1] - 1]), lw=1)
	gca.add_line(line)
	edges = np.append(edges, line)

(12,)
<class 'numpy.ndarray'>
init: ready
lambda: ready


## ドラッグした時の座標のサンプル

In [18]:
X_sample = 3 * np.random.random_sample((100, 1)) - 1.5
Y_sample = 3 * np.random.random_sample((100, 1)) - 1.5

## 以前のバージョンにおける lambdify の計算時間

In [19]:
varLambdify = (a1, b1, c1, a2, b2, c2, s, t)
func = sp.Matrix.norm(f)
# V: vector symbol, Vv:np.ndarray
def subV(e, V, Vv):
    for i in range(len(Vv)):
            e = e.subs(V[i,0], Vv[i])
    return e
# Vs, Vvs : list
def subV_mul(e,Vs,Vvs):
    if len(Vvs) > 0:
        if isinstance(Vvs[0], np.ndarray):
            if(len(Vvs[0])>1):
                return subV_mul(subV(e,Vs[0],Vvs[0]),Vs[1:],Vvs[1:])
            else:
                return subV_mul(e.subs(Vs[0],Vvs[0]),Vs[1:],Vvs[1:])
        else:
            return subV_mul(e.subs(Vs[0],Vvs[0]),Vs[1:],Vvs[1:])
    return e

import time
start = time.time()

for i in range(10):
    lamlam = lambdify(varLambdify,subV_mul(func,[x2_s,y2_s,P_i,E1,E2],[X_sample[i],Y_sample[i],P[0],e1,e2]),'numpy')

elapsed_time = time.time() - start
print("elapsed_time: " + str(elapsed_time) + "[sec]")
print("average_time: " + str(elapsed_time / 10) + "[sec]")

elapsed_time: 708.3559119701385[sec]
average_time: 70.83559119701385[sec]


## 現在の計算における lambdify の計算時間

In [20]:
import time

start = time.time()

for i in range(100):
    lam_f = lambdify(var, func, 'numpy')

elapsed_time = time.time() - start
print ("elapsed_time: " + str(elapsed_time) + "[sec]")
print("average_time: " + str(elapsed_time / 100) + "[sec]")

elapsed_time: 343.51510787010193[sec]
average_time: 3.4351510787010193[sec]


## opt.minimize(L-BFGS=B)の計算時間

In [21]:
import time

start = time.time()

for i in range(100):
    def lam(x2, y2, p, e_1, e_2):
        return lambda a1,b1,c1,a2,b2,c2,t,s: \
            lam_f(x2, y2, p, e_1, e_2, a1, b1, c1, a2, b2, c2, t, s)
    f2 = lam(X_sample[i], Y_sample[i], P[0].reshape(d,1), e1, e2)
    def g(args): return f2(*args)
    res = opt.minimize(g, arr, method='L-BFGS-B')

elapsed_time = time.time() - start
print ("elapsed_time: " + str(elapsed_time) + "[sec]")
print("average_time: " + str(elapsed_time / 100) + "[sec]")

elapsed_time: 4.132338047027588[sec]
average_time: 0.04132338047027588[sec]


## 基底更新の計算時間

In [22]:
resS = np.array([])
for i in range(100):
    def lam(x2, y2, p, e_1, e_2):
        return lambda a1,b1,c1,a2,b2,c2,t,s: \
            lam_f(x2, y2, p, e_1, e_2, a1, b1, c1, a2, b2, c2, t, s)
    f2 = lam(X_sample[i], Y_sample[i], P[0].reshape(d,1), e1, e2)
    def g(args): return f2(*args)
    res = opt.minimize(g, arr, method='L-BFGS-B')
    resS = np.append(resS,res)

In [31]:
import time

start = time.time()

for i in range(100):
	e1 = resS[i].x[0] * temp1 + resS[i].x[1] * temp2 + resS[i].x[2] * P[0].reshape(d,1)
	e2 = resS[i].x[3] * temp1 + resS[i].x[4] * temp2 + resS[i].x[5] * P[0].reshape(d,1)
	temp1 = e1
	temp2 = e2

elapsed_time = time.time() - start
print ("elapsed_time: " + str(elapsed_time) + "[sec]")
print("average_time: " + str(elapsed_time / 100) + "[sec]")

elapsed_time: 0.0016651153564453125[sec]
average_time: 1.6651153564453126e-05[sec]


## 座標の計算時間(np.dot)

In [32]:
e1S = []
e2S = []
for i in range(100):
	e1S.append(resS[i].x[0] * temp1 + resS[i].x[1] * temp2 + resS[i].x[2] * P[0].reshape(d,1))
	e2S.append(resS[i].x[3] * temp1 + resS[i].x[4] * temp2 + resS[i].x[5] * P[0].reshape(d,1))

In [33]:
import time

start = time.time()

for k in range(100):
    e1 = e1S[i]
    e2 = e2S[i]
    update_points()

elapsed_time = time.time() - start
print ("elapsed_time: " + str(elapsed_time) + "[sec]")
print("average_time: " + str(elapsed_time / 100) + "[sec]")

elapsed_time: 0.003598928451538086[sec]
average_time: 3.598928451538086e-05[sec]


## ノードエッジの更新時間

In [34]:
import time

start = time.time()

for k in range(100):
    for i in np.arange(n):
        nodes[i].center = (Xs[i], Ys[i])
    for i in range(len(edge)):
        edges[i].set_xdata((Xs[edge[i,0] - 1], Xs[edge[i,1] - 1]))
        edges[i].set_ydata((Ys[edge[i,0] - 1], Ys[edge[i,1] - 1]))

elapsed_time = time.time() - start
print ("elapsed_time: " + str(elapsed_time) + "[sec]")
print("average_time: " + str(elapsed_time / 100) + "[sec]")

elapsed_time: 0.024055957794189453[sec]
average_time: 0.00024055957794189454[sec]
